https://scikit-learn.org/1.1/modules/linear_model.html#logistic-regression

In [1]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)
(X.shape, y.shape)

((569, 30), (569,))

In [2]:
[(X[0], y[0]), (X[568], y[568])]

[(array([1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
         3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
         8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
         3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
         1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01]),
  0),
 (array([7.760e+00, 2.454e+01, 4.792e+01, 1.810e+02, 5.263e-02, 4.362e-02,
         0.000e+00, 0.000e+00, 1.587e-01, 5.884e-02, 3.857e-01, 1.428e+00,
         2.548e+00, 1.915e+01, 7.189e-03, 4.660e-03, 0.000e+00, 0.000e+00,
         2.676e-02, 2.783e-03, 9.456e+00, 3.037e+01, 5.916e+01, 2.686e+02,
         8.996e-02, 6.444e-02, 0.000e+00, 0.000e+00, 2.871e-01, 7.039e-02]),
  1)]

In [3]:
import getpass
import mlflow
from liga.sklearn.mlflow import log_model
from sklearn.linear_model import LogisticRegression

mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
model = LogisticRegression()
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model.fit(X, y)
    registered_model_name = f"{getpass.getuser()}_sklearn_log_r"
    log_model(model, registered_model_name=registered_model_name)


/home/da/.cache/pants/named_caches/pex_root/venvs/s/378f4125/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/da/.cache/pants/named_caches/pex_root/venvs/s/378f4125/venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/03/29 13:10:32 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'da_sklearn_l

In [4]:
from example import spark

2023-03-29 13:10:33,110 INFO Rikai (__init__.py:121): setting spark.sql.extensions to net.xmacs.liga.spark.RikaiSparkSessionExtensions
2023-03-29 13:10:33,111 INFO Rikai (__init__.py:121): setting spark.driver.extraJavaOptions to -Dio.netty.tryReflectionSetAccessible=true
2023-03-29 13:10:33,111 INFO Rikai (__init__.py:121): setting spark.executor.extraJavaOptions to -Dio.netty.tryReflectionSetAccessible=true
2023-03-29 13:10:33,112 INFO Rikai (__init__.py:121): setting spark.jars to https://github.com/komprenilo/liga/releases/download/v0.3.0/liga-spark321-assembly_2.12-0.3.0.jar
23/03/29 13:10:33 WARN Utils: Your hostname, tubi resolves to a loopback address: 127.0.1.1; using 192.168.31.32 instead (on interface wlp0s20f3)
23/03/29 13:10:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For Spar

## Create the model using the registered MLflow uri

In [5]:
from liga.mlflow import CONF_MLFLOW_TRACKING_URI

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m LOCATION 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)

+----------------+------+--------------------------+-------+
|name            |plugin|uri                       |options|
+----------------+------+--------------------------+-------+
|mlflow_sklearn_m|      |mlflow:///da_sklearn_log_r|       |
+----------------+------+--------------------------+-------+



## Predict using the registered Rikai model

In [6]:
from liga.numpy.sql import literal
 
result = spark.sql(f"""
select ML_PREDICT(mlflow_sklearn_m, {literal(X[0])})
"""
)

result.printSchema()
result.toPandas()

root
 |-- mlflow_sklearn_m: integer (nullable = true)



,mlflow_sklearn_m
0,0


In [7]:
result = spark.sql(f"""
select ML_PREDICT(mlflow_sklearn_m, {literal(X[568])})
"""
)

result.printSchema()
result.toPandas()

root
 |-- mlflow_sklearn_m: integer (nullable = true)



,mlflow_sklearn_m
0,1
